In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
from MILWRM.ST import *
from mycolorpy import colorlist as mcp

In [ ]:
# make output directories
import os
if not os.path.exists("ST_out"):
    os.mkdir("ST_out")

if not os.path.exists("ST_out/CytoTRACE/"):
    os.mkdir("ST_out/CytoTRACE/")

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=180)
sc.settings.figdir = "ST_out/CytoTRACE/"
import seaborn as sns; sns.set_style("white")

---
### Read in key dataframe with sample information

In [ ]:
sample_key = pd.read_csv("../resources/ST/visium_sample_key.csv", index_col=0)

In [ ]:
%%time
for s in sample_key.index[:2]:
    print("Starting {}:".format(s), end="\n")
    a = sc.read("../data/ST/{}_master.h5ad".format(s))  # read in anndata
    
    if "CytoTRACE" in a.obs.columns:
        a.obs.drop(columns=["CytoTRACE"], inplace=True)
        
    # get CytoTRACE scores per spot
    print("Reading CytoTRACE from ../step1/ST_out/CytoTRACE/{}_CytoTRACE.csv".format(sample_key.loc[s, "sample_key_short"]))
    tmp = pd.read_csv("../step1/ST_out/CytoTRACE/{}_CytoTRACE.csv".format(sample_key.loc[s, "sample_key_short"]), index_col=0)
    tmp.rename(columns={"x":"CytoTRACE"},inplace=True)
    a.obs = a.obs.merge(tmp, left_index=True, right_index=True, how="left")
    
    # plot CytoTRACE overlay
    p = assemble_pita(
        a,
        features="CytoTRACE",
        cmap="viridis",
        use_rep="obs",
        histo="hires_trim",
        label="CytoTRACE",
        save_to="ST_out/CytoTRACE/{}_CytoTRACE.png".format(s),
    )

    # save to master anndata object
    print("\tSaving to {}".format("../data/ST/{}_master.h5ad".format(s)), end="\n\n")
    a.write("../data/ST/{}_master.h5ad".format(s), compression="gzip")